In [1]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.init as init
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import time
from PIL import Image
import torch.nn.functional as F
import torchvision.datasets as datasets
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score,roc_curve, auc
import seaborn as sns

In [2]:
# Load train data
#the running of this seperate file needs the revision of relative path

train_label_path = './Datasets/celeba/labels.csv'
train_img_path = './Datasets/celeba/img/'
test_label_path = './Datasets/celeba_test/labels.csv'
test_img_path = './Datasets/celeba_test/img/'

train_label = pd.read_csv(train_label_path, sep = "\t")  # read csv file
train_label.loc[train_label['gender'] == -1, 'gender'] = 0 
y_train = train_label['gender']
img_name1 = train_label['img_name']

x_train = []
for name in img_name1:
    img_path =  train_img_path + name # get path based on image name
    img = Image.open(img_path)
    x_train.append(img)  # add pic to x_train

#split the validation dataset from test dataset
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=20)
    
#load test and vadilation data
data_test = pd.read_csv(test_label_path, sep = "\t")  # read csv file
data_test.loc[data_test['gender'] == -1, 'gender'] = 0 
y_test = data_test['gender']
img_name2 = data_test['img_name']

x_test=[]
for name in img_name2:
    img_path = test_img_path + name # get path based on image name
    img = Image.open(img_path)
    x_test.append(img)  # add pic to x_train

print("Dataset prepared")

Dataset prepared


In [3]:
# Transform
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=15),
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize([0.412,0.412,0.412], [0.245,0.245,0.245])
])
transform_test = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize([0.412,0.412,0.412], [0.245,0.245,0.245])
])
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y.to_numpy())
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else: 
            return X
        
batch_size = 128
train_set = ImgDataset(x_train, y_train, transform=transform_train)
val_set = ImgDataset(x_val, y_val, transform=transform_test)
test_set = ImgDataset(x_test, transform=transform_test)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
print("Dataset loaded")
#print(len(train_set))
#print(len(val_set))
#print(len(test_set))

Dataset loaded
4000
1000
1000


In [4]:
class Classifier(nn.Module):
    """
    CNN is used for task A1, with multiple layers
    """
    def __init__(self, num_classes = 2):
        super().__init__()
        self.num_classes = num_classes
        #convolutional layer
        self.conv2d = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        # Fully connected layer
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=128 * 8 * 8, out_features=256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.4, inplace=False),
            nn.Linear(in_features=256, out_features=num_classes)
        )
        # Initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)
               
    def forward(self, input):
        output = self.conv2d(input)
        output = self.fc(output)
        return output

print("Model completed")

Model completed


In [5]:
''' Testing '''
print("Testing")
print("...")
best_model = Classifier().cpu()
best_model.load_state_dict(torch.load("./A1/cnn_model.pt"))
best_model.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = best_model(data.cpu())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)
            
print('Accuracy on test set: '+str(accuracy_score(y_test,prediction)))
print(classification_report(y_test,prediction))
# Confusion Matrix
cm = confusion_matrix(y_test,prediction)
df_cm = pd.DataFrame(cm)
fig, ax = plt.subplots(figsize=(6,4)) 
ax = sns.heatmap(df_cm, 
                annot=True, 
                fmt=".20g", 
                linewidths=2, 
                square=True
                )

ax.set_xlabel('True', family='Arial')
ax.set_ylabel('Predict', family='Arial')
ax.set_title('Confusion Matrix', family='Arial')
plt.tight_layout()
plt.savefig('A1_CNN.png', dpi=300)

Testing
...
Accuracy on test set: 0.96


'\n# Confusion Matrix\ncm = confusion_matrix(y_test,prediction)\ndf_cm = pd.DataFrame(cm)\nfig, ax = plt.subplots(figsize=(6,4)) \nax = sns.heatmap(df_cm, \n                annot=True, \n                fmt=".20g", \n                linewidths=2, \n                square=True\n                )\n\nax.set_xlabel(\'True\', family=\'Arial\')\nax.set_ylabel(\'Predict\', family=\'Arial\')\nax.set_title(\'Confusion Matrix\', family=\'Arial\')\nplt.tight_layout()\nplt.savefig(\'Heatmap_CNN.png\', dpi=300)\n'